### End to End Demo for Cleanlab Studio Python API

1) Initialize Studio object with your API key

In [2]:
from cleanlab_studio import Studio

api_key = "<insert_api_key>"
studio = Studio(api_key)

2. Load dataset as pandas dataframe

In [3]:
import pandas as pd

df = pd.read_csv("https://s.cleanlab.ai/amazon-text-demo.csv")
print(df.head())

   index                                        review_text        asin  stars
0      0  This magazine was great for the times but as w...  B00005N7P0    ⭐⭐⭐
1      1  We ordered this magazine for our grandson (the...  B00005N7P0  ⭐⭐⭐⭐⭐
2      2  I didn't receive a full year.  I only receive ...  B00005N7P0     ⭐⭐
3      3  I was hoping for more technical than what was ...  B00005N7P0     ⭐⭐
4      4  I only received one copy of the mag so I could...  B00005N7P0     ⭐⭐


3. Upload dataset to Studio

In [4]:
dataset_id = studio.upload_dataset(df, "Amazon Reviews")
dataset_id

Uploading dataset...: 100%|██████████|
Generating schema...: 100%|██████████|
Ingesting Dataset...: 100%|██████████|


'e46b457d112c4e78b4eec0c3a54abdea'

4. Create Project

In [5]:
project_id = studio.create_project(dataset_id, "Amazon Review Project", "text", label_column="stars", text_column="review_text", model_type="fast")
project_id

'60b1f04550b44bcfbc92e660cad576a6'

5. Get cleanset ID for project

In [6]:
cleanset_id = studio.get_latest_cleanset_id(project_id)
cleanset_id

'fd6acdb32cc14023862a598937e26c7d'

6. Poll for cleanset status until cleanset is ready

In [7]:
studio.poll_cleanset_status(cleanset_id)

Cleanset Progress: \ Step 5/5, Ready...          


True

7. Download cleanlab columns

In [8]:
cleanlab_columns_df = studio.download_cleanlab_columns(cleanset_id)
cleanlab_columns_df.head()

,index,cleanlab_issue,cleanlab_label_quality,cleanlab_suggested_label,cleanlab_clean_label
0,0,False,0.743142,None,None
1,1,True,0.312316,⭐⭐⭐⭐,None
2,2,True,0.281833,⭐⭐⭐,None
3,3,False,0.536544,None,None
4,4,False,0.401961,None,None
